In [4]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
import pandas as pd
import datetime
import pickle

In [5]:
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(keyword,start_pg,end_pg,target_date):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+keyword+"&sort=0&photo=0&field=0&pd=3&ds="+target_date+"&de="+target_date+"&cluster_rank=51&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20220310to20220310,a:all&start="+str(page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            try:
                page = makePgNum(i)
                url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+keyword+"&sort=0&photo=0&field=0&pd=3&ds="+target_date+"&de="+target_date+"&cluster_rank=51&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20220310to20220310,a:all&start="+str(page)
                urls.append(url)
            except:
                break
        return urls

In [72]:
target_day=datetime.datetime(2018,12,31)
t_date=[]
t_date2=[]
#
for i in range(730):
    t1_date=target_day-datetime.timedelta(days=i)
    t1_date=t1_date.strftime('%Y.%m.%d')
    t_date.append(t1_date)

    t2_date=target_day-datetime.timedelta(days=i)
    t2_date=t2_date.strftime('%Y%m%d')
    t_date2.append(t2_date)

In [73]:
target_day

datetime.datetime(2018, 12, 31, 0, 0)

In [74]:
keywords = ['탄소+저장+안전','탄소+지중+안전','탄소+지중+지진','탄소+포항+지진','탄소+시민','탄소+촉발+지진']
naver_urls_0=[]
naver_urls_1=[]
naver_urls_2=[]
naver_urls_3=[]
naver_urls_4=[]
naver_urls_5=[]

for g in t_date:
    for k,keyword in enumerate(keywords):
        search_urls = makeUrl(keyword,1,3,g)
        ## selenium으로 navernews만 뽑아오기##
        driver = webdriver.Chrome('C://chromedriver.exe')
        driver.implicitly_wait(3)
        
        # selenium으로 검색 페이지 불러오기 #
        count = 0 # 핸들러 변수(창 바꿀때 사용)

        for i in search_urls:
            driver.get(i)
            time.sleep(1) #대기시간 변경 가능

            # 네이버 기사 눌러서 제목 및 본문 가져오기#
            # 네이버 기사가 있는 기사 css selector 모아오기
            a = driver.find_elements_by_css_selector('a.info')

            # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
            for j in a:
                count+=1
                j.click()

                # 현재탭에 접근
                driver.switch_to_window(driver.window_handles[count-1])
                time.sleep(5) #대기시간 변경 가능

                # 네이버 뉴스 url만 가져오기

                try:
                    url = driver.current_url
                    print(url)

                    if "news.naver.com" in url:
                        exec(f"naver_urls_{k}.append(url)")
                    else:
                        pass

                    # 다시처음 탭으로 돌아가기(매우 중요!!!)
                    driver.switch_to_window(driver.window_handles[0])
                except:
                    # 다시처음 탭으로 돌아가기(매우 중요!!!)
                    driver.switch_to_window(driver.window_handles[0])
                    pass
        driver.quit()
        time.sleep(1)
    time.sleep(1)

C:\Users\jmkim\anaconda3\envs\jam\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: use driver.switch_to.window instead


https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%ED%83%84%EC%86%8C+%EC%A0%80%EC%9E%A5+%EC%95%88%EC%A0%84&sort=0&photo=0&field=0&pd=3&ds=2018.12.31&de=2018.12.31&cluster_rank=51&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20220310to20220310,a:all&start=1


C:\Users\jmkim\anaconda3\envs\jam\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: use driver.switch_to.window instead


https://www.mk.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=009&aid=0004280722
https://www.seoul.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=081&aid=0002967186
https://www.hankyung.com/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=015&aid=0004070770
https://www.newspim.com/
http://www.industrynews.co.kr/
http://www.industrynews.co.kr/


In [71]:
naver_urls_5

NameError: name 'naver_urls_5' is not defined

In [59]:
# ConnectionError방지
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

titles = []
contents=[]
dates=[]
companys=[]
pic_contents=[]

#### naver_urls_0 ~ naver_urls_4 는 아래 filenames 키워드 순서대로 뉴스기사url 주소들이 들어있음
#### 제법 오래 걸릴 수 있음
filenames = ['탄소_저장_안전','탄소_지중_안전','탄소_지중_지진','탄소_포항_지진','탄소_시민','탄소_촉발_지진']
naver_url_list = [naver_urls_0,naver_urls_1,naver_urls_2,naver_urls_3,naver_urls_4]

for filename,naver_urls in zip(filenames,naver_url_list):
    for i in naver_urls:
        original_html = requests.get(i,headers=headers)
        html = BeautifulSoup(original_html.text, "html.parser")
        # 검색결과확인시
        #print(html)
        
        #뉴스 제목 가져오기
        title = html.select("div.content > div.article_header > div.article_info > h3")
        # list합치기
        title = ''.join(str(title))
        # html태그제거
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1,repl='',string=title)
        titles.append(title)

        #뉴스 본문 가져오기

        content = html.select("div.content > div#articleBody > div#articleBodyContents")

        # list합치기
        content = ''.join(str(content))
        
        #html태그제거 및 텍스트 다듬기
        content = re.sub(pattern=pattern1,repl='',string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2,'')
        content = re.sub(pattern=pattern1,repl='',string=content)
        
        # contents.append(content)

        content=list([content])
        contents.append(content)

        # 뉴스 언론사 가져오기
        company = html.find('img')['alt']
        companys.append(company)


        # 뉴스 날짜 가져오기
        date = html.select("div.article_info > div.sponsor > span.t11")
        # list합치기
        date = ''.join(str(date))
        #html태그제거 및 텍스트 다듬기
        # date = re.sub(pattern=pattern1,repl='',string=date)

        dates.append(date)

        # 뉴스 그림글 가져오기
        pic_content = html.select("em.img_desc")
        # list합치기
        pic_content = ''.join(str(pic_content))
        #html태그제거 및 텍스트 다듬기
        pic_content = re.sub(pattern=pattern1,repl='',string=pic_content)

        pic_contents.append(pic_content)

    titles=pd.DataFrame(titles)
    contents=pd.DataFrame(contents)
    pic_contents=pd.DataFrame(pic_contents)
    companys=pd.DataFrame(companys)
    dates=pd.DataFrame(dates)

    news=pd.concat([titles,contents,pic_contents,companys,dates],axis=1)
    news.columns=['기사제목','기사내용','그림글','언론사','기사날짜']
    news.to_csv('./naver_news_{}.csv'.format(filename),index=False,encoding='utf-8-sig')      # 뉴스 엑셀로 저장

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [56]:
title='[서울 코로나 확진 8만1997명…중환자 병상 가동률 63.3%]'
title=list([title])
title

['[서울 코로나 확진 8만1997명…중환자 병상 가동률 63.3%]']

In [57]:
titles.append(title)

,0
0,[도크 가득 채운 LNG선…칼바람에도 멈추지않는 용접불꽃]
1,[무엇이 김용균들을 죽음으로 내모나]
2,[[신년 기획] '눈물·분노·안타까운 죽음'…2018년 월별 사건사고 캘린더]
3,[[블록체인 2018] 블록체인 공공사업 2019년도 설명회 총정리]
4,[[기고] 사용후 핵연료에 대한 실체적 접근 필요]
...,...
85,"[삼성전자 이산화탄소 누출사고 원인 ""소화설비 배선 오인 절단"" ]"
86,[창원시 차세대 먹거리 수소 산업 육성에 박차 ]
87,[ 디지털 강소국 에스토니아 대통령 방한…“정부가 개인정보 안전 느낌 줘야” ]
88,"[ [단독] 삼성, ‘CO₂ 누출 사망’ 소화설비 43개 사업장에 아직 그대로 ]"


In [41]:
title

'[서울 코로나 확진 8만1997명…중환자 병상 가동률 63.3%]'

In [22]:
a=[]
a.append(title)

In [23]:
a.append(title)

['[서울 코로나 확진 8만1997명…중환자 병상 가동률 63.3%]']

In [67]:
naver_urls_00=pd.DataFrame(naver_urls_0)
naver_urls_01=pd.DataFrame(naver_urls_1)
naver_urls_02=pd.DataFrame(naver_urls_2)
naver_urls_03=pd.DataFrame(naver_urls_3)
naver_urls_04=pd.DataFrame(naver_urls_4)

naver_urls_00.to_csv('./urldata/naver_url_0.csv',encoding='utf-8-sig')
naver_urls_01.to_csv('./urldata/naver_url_1.csv',encoding='utf-8-sig')
naver_urls_02.to_csv('./urldata/naver_url_2.csv',encoding='utf-8-sig')
naver_urls_03.to_csv('./urldata/naver_url_3.csv',encoding='utf-8-sig')
naver_urls_04.to_csv('./urldata/naver_url_4.csv',encoding='utf-8-sig')